# Lien entre la base souscription et la base le furet

- Code Original : Marouan 
- Code original pour lecture Le Furet : Clémentine
- Révisé : Guillaume Gorge 2/5/23
- Révisé Marouan : 23 06 2023




## 1. Initialisation Phase


In [994]:
# variables indiquant si on relit ou non toute la base

# generation_source = 'Aggregator' 'Portal'

In [995]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import math
from plotly import graph_objects as go
import json 
import seaborn as sns
import pandas_profiling
import sys


In [996]:
## Python directory
### <--------------------------- TO BE CHANGED ---------------------------> ### 
PYTHON_DIRECTORY = 'https://github.com/ornikar/pricing-insurance/Shared Notebooks'
PYTHON_DIRECTORY = '/Users/marouan.korfed/Desktop/GitHub/2023 Ornikar Pricing/pricing-insurance/Python Official Library/'
 
LATEX_MAIN_DIRECTORY = '/Users/marouan.korfed/Desktop/GitHub/2023 Ornikar Pricing/pricing-insurance/Latex/OrnikarInsuranceAutomaticResults/'
LATEX_DIRECTORY = '/Users/marouan.korfed/Desktop/GitHub/2023 Ornikar Pricing/pricing-insurance/Latex/OrnikarInsuranceAutomaticResults/Dashboard_Latex/'

RESULTS_DIRECTORY = '/Users/marouan.korfed/Desktop/GitHub/2023 Ornikar Pricing/pricing-insurance/Results/'
### <--------------------------- TO BE CHANGED ---------------------------> ### 


sys.path.insert(1, PYTHON_DIRECTORY)


from importlib import reload  # Python 3.4+
import OrnikarDataFormat
import TransformedDataReading
import LinkQuotations
import LeFuretLibrary
import GraphicStatLibrary

# instruction pour forcer la relecture d'une base (utile quand on code ;) 
# LeFuretLibrary = reload(LeFuretLibrary)
# GraphicStatLibrary = reload(GraphicStatLibrary)


In [997]:
# Définition du mode de lecture / écriture des base : 
ModeGlobal = True # True or False : True pour tout lire
ModeSpeed = 'Full' # 'Full' means to keep all variables at the expense of Speed. Put 'Quick' instead
periodLeFuret =  '2023-03' # mois le Furet à lire
if ModeGlobal and ModeSpeed == 'Full':
    print("Full reading of BigQuery and all DataBases")
    mode = 'Google' 
    SizeVariablesKeptGW = 'Full' # We keep all the variables of GW except var_to_suppress.
elif ModeGlobal:
    print("Fast Mode through limited selection of variables - good for Clementine")
    mode = 'Google' 
    SizeVariablesKeptGW = 'Quick' # We keep all the variables of GW except var_to_suppress.
else: 
    mode = 'ReadLocalBase'
    SizeVariablesKeptGW = 'Full'


Full reading of BigQuery and all DataBases


In [998]:

# Import the data base

### <--------------------------- TO BE CHANGED ---------------------------> ### 
BASE_PATH = '/Users/marouan.korfed/Desktop/GitHub/2023 Ornikar Empreinte/Data' 
BASE_PATH_LEFURET = '/Users/marouan.korfed/Desktop/GitHub/2023 Coherence Tarifaire/Data' 
BASE_PATH_RES = RESULTS_DIRECTORY
### <--------------------------- TO BE CHANGED ---------------------------> ### 


In [999]:
## Variables to Keep Guidewire

# Liste Clémentine
var_process = ['account_id',  'account_number', 'submission_created_at', 'bind_date',  'job_id',  'job_number', 'policy_id',  'policy_number', 'tariff_viewed', 'wakam_called', 'tariff_accepted', 'wakam_pricing_version',
                'generation_source', 'online_offline' ,'ph_email']
var_risk = ['ph_email', 'loss_history_type', 'recommended_package', 'package_subtype', 'md_age', 'md_license_duration', 'vh_age',  'md_prefix',  'md_crm_coefficient', 'md_license_country', 'vh_sra_code', 'vh_license_plate','vh_annual_km','PACKAGE', ]  
# if SizeVariablesKeptGW = 'Full' alors à ces variables sont ajoutées TOUTES les variables sauf var_to_suppress
var_to_suppress = ['vh_sra_code','Unnamed: 0', 'insurer_name','submission_creation_user_name','update_user_id','ph_first_name','ph_last_name','ph_email','ph_phone'
] # vh_sra_code est en doublon

# Liste Guillaume 
var_process = ['account_id',  'account_number', 'submission_created_at', 'bind_date',  'job_id',  'job_number', 'policy_id',  'policy_number', 'tariff_viewed', 'wakam_called', 'tariff_accepted', 'policy_status', 'wakam_pricing_version',
                'generation_source', 'online_offline','ph_email' ]
var_risk = [ 'loss_history_type', 'recommended_package', 'package_subtype', 'md_age', 'md_license_duration', 'vh_age',  'md_prefix',  'md_crm_coefficient', 'md_license_country', 'vh_sra_code', 'vh_license_plate','vh_annual_km','PACKAGE', ]  

# suppression 'ph_gender' et remplacement par md_prefix

var_premium = ['total_premium_rpt','total_cost_rpt']


In [1000]:
# Variable to Keep LeFuret

columnstoKeep = ['IdClient', 'SRA_Groupe', 'SRA_Classe', 'VEH_Antecedent',
       'premium_TTC', 'premium_HT', 'premium_RCC_TTC', 'premium_RCM_TTC',
       'premium_DRX_TTC', 'premium_PROPIL_TTC', 'premium_BDG_TTC',
       'premium_VLX_TTC', 'premium_INC_TTC', 'premium_CATNAT_TTC',
       'premium_CATECH_TTC', 'premium_EVCLM_TTC', 'premium_DOM_TTC', 'PP_RC',
       'PP_VOL', 'PP_INC', 'PP_BDG', 'PP_DOM', 'PP_SDC','message','ph_email']



columnstoKeepSmall = [ 'IdClient',
       'premium_TTC', 'premium_HT', 'premium_RCC_TTC', 'premium_RCM_TTC',
       'premium_DRX_TTC', 'premium_PROPIL_TTC', 'premium_BDG_TTC',
       'premium_VLX_TTC', 'premium_INC_TTC', 'premium_CATNAT_TTC',
       'premium_CATECH_TTC', 'premium_EVCLM_TTC', 'premium_DOM_TTC','message']

columnstoKeepSmall =['IdClient','premium_TTC','premium_HT','PP_RC','PP_VOL', 'PP_INC', 'PP_BDG', 'PP_DOM', 'PP_SDC','message'] 
# La prime Furet est TTC, premium HT est conservée pour le S/C

columnsPremium = ['premium_TTC', 'premium_HT', 'premium_RCC_TTC', 'premium_RCM_TTC',
       'premium_DRX_TTC', 'premium_PROPIL_TTC', 'premium_BDG_TTC',
       'premium_VLX_TTC', 'premium_INC_TTC', 'premium_CATNAT_TTC',
       'premium_CATECH_TTC', 'premium_EVCLM_TTC', 'premium_DOM_TTC','message']
columnsPremiumRC = [c+'_RC' for c in columnsPremium]
columnsPremiumRCP = [c+'_RCP' for c in columnsPremium]
columnsPremiumTR = [c+'_TR' for c in columnsPremium]

In [1001]:
# Base le Furet méthode 2 : description automatique et suppression des variables non pertinentes.
# voir méthode que l'on retient au final

# Attention, clémentine, pour toi, faire un filtre sur [['custom_qr_id', 'couverture_demandée', 'mediane', 'annee_naissance', 'segment']]
featureLF_a_conserver = [['custom_qr_id', 'couverture_demandée', 'mediane', 'segment','md_age']] 

features_a_supprimer =['custom_qr_id','Unnamed: 0','Unnamed: 0.1','assureur','id','mediane', 'moyenne', 'position_vue', 'prix', 'id_externe_tarif', 'couverture','custom_offer_id',
                       'DTA_Incluse',	'DTA_Franchise_Montant',	'VI_Incluse',	'VI_Franchise_Montant',	'BDG_Incluse',	'BDG_Franchise_Montant',	'ASS_Panne_Incluse',	'ASS_Panne_Franchise_Montant',	'ASS_Accident_Incluse',	'ASS_Accident_Franchise_Montant',	'VR_Accident_Incluse',	'GC_Incluse',
                     	'custom_offer_id','PositionMediane','Position5HorsMediane','position_mer','devis','rappel','souscription']  
# un certain nombre de variables buguent car plusieurs formats. On les enlève dans un premier temps
# ! TODO il faudra essayer de les reintégrer
features_trop_large = ['code_postal','code_postal_travail','auto_sec_his_assure_actuel',
                       'auto_veh_tra_france','auto_pri_inf_conjoint_permis','auto_pri_inf_conjoint_autre_veh','auto_pri_his_sin_1_tiers','auto_pri_his_sin_2_tiers',
                       'auto_pri_his_sin_3_tiers','auto_pri_his_sin_4_tiers','auto_pri_his_sin_5_tiers',
                       'auto_pri_his_sin_corp_1_enquete','auto_pri_his_sin_corp_2_enquete','auto_pri_his_sin_corp_3_enquete','auto_pri_his_sin_corp_4_enquete','auto_pri_his_sin_corp_5_enquete',' ',
                       'auto_pri_his_sin_corp_1_prison', 'auto_pri_his_sin_corp_1_prison', 'auto_pri_his_sin_corp_2_prison','auto_pri_his_sin_corp_3_prison','auto_pri_his_sin_corp_4_prison','auto_pri_his_sin_corp_5_prison',
                       'auto_ass_dtl_souscription']
# auto_veh_garage_nt_departement_value était avant enlevé


In [1002]:
### Variable Wakam
# Faire le Tri
VariableSRAWakam= ['SRA_CodeEnergie	SRA_PtacENkg','SRA_PuissanceReelleMaxiCEE',	'VEH_RatioWeightPower',	'SRA_Suspension	SRA_Alimentation',	'SRA_VitesseMaxi',	'SRA_TypeFreins',	'SRA_Carrosserie',	'VEH_Carrosserie_FOCUS',	'SRA_Transmission',	'SRA_NombrePlaces',	'SRA_ClasseReparationActuelle']
VariableSRAWakam= ['SRA_PuissanceReelleMaxiCEE','VEH_RatioWeightPower']



In [1003]:
#Table de référence pour les libellés correspondant à une couverture donnée
var = {
    "libelle_import": ["RC", "RC+", "TR"], 
    "sexy_title": ["1. Tiers", "2. Tiers Etendue", "3. Tous Risques"],
    "2023-02" : ['All_Quote_Priced_02_2023_RC','All_Quote_Priced_02_2023_RC+','All_Quote_Priced_02_2023_TR'],
    "2023-03" : ['All_Quotes_RC_03_2023','All_Quotes_RC+_03_2023','All_Quotes_TR_03_2023'],
    }

couverture = ["tiers", "vol_incendie", "tous_risques"]
dico_couv = pd.DataFrame(var, index = couverture)
dico_couv

,libelle_import,sexy_title,2023-02,2023-03
tiers,RC,1. Tiers,All_Quote_Priced_02_2023_RC,All_Quotes_RC_03_2023
vol_incendie,RC+,2. Tiers Etendue,All_Quote_Priced_02_2023_RC+,All_Quotes_RC+_03_2023
tous_risques,TR,3. Tous Risques,All_Quote_Priced_02_2023_TR,All_Quotes_TR_03_2023


## 2. Reading DataBases

In [1004]:
## Read either data base located on the computer (mode = 'ReadLocalBase') or through Google Big Query directly with the update of local data base (option = "Google")

df, df_previous_claims, df_previous_insurers, df_previous_terminations, df_previous_license_cancellations_suspensions  = TransformedDataReading.GoogleAllDataBaseReading(mode=mode, BASE_PATH =BASE_PATH)
#df, df_previous_claims, df_previous_insurers, df_previous_terminations, df_previous_license_cancellations_suspensions  = TransformedDataReading.GoogleAllDataBaseReading(mode='Google', BASE_PATH =BASE_PATH)

/Users/marouan.korfed/miniconda3/lib/python3.10/site-packages/google/auth/_default.py:78: UserWarning: Your application has authenticated using end user credentials from Google Cloud SDK without a quota project. You might receive a "quota exceeded" or "API not enabled" error. See the following page for troubleshooting: https://cloud.google.com/docs/authentication/adc-troubleshooting/user-creds. 
  warnings.warn(_CLOUD_SDK_CREDENTIALS_WARNING)
/Users/marouan.korfed/miniconda3/lib/python3.10/site-packages/google/auth/_default.py:78: UserWarning: Your application has authenticated using end user credentials from Google Cloud SDK without a quota project. You might receive a "quota exceeded" or "API not enabled" error. See the following page for troubleshooting: https://cloud.google.com/docs/authentication/adc-troubleshooting/user-creds. 
  warnings.warn(_CLOUD_SDK_CREDENTIALS_WARNING)
/Users/marouan.korfed/miniconda3/lib/python3.10/site-packages/google/auth/_default.py:78: UserWarning: You

In [1014]:
(df['ph_first_name'].isna()).sum()

115

In [1016]:
# Formatting Data 
OrnikarDataFormat = reload(OrnikarDataFormat)
dfFormat = OrnikarDataFormat.FormatGuideWire(df)
print('Number of policies UW : '+str(dfFormat['tariff_accepted'].sum()))

['submission_created_at', 'job_close_date', 'bind_date', 'issue_date', 'cancellation_date', 'original_effective_date', 'period_start', 'period_end', 'creation_date', 'update_date', 'price_computed_at', 'written_date', 'policy_period_start', 'policy_period_end']
Number of policies UW : 2724


In [1017]:
# Commentaire Marouan : je l'ajoute ici en solution temporaire car ne fonctionne pas si dans OrnikarDataFormat (pb sur les dates à régler par la suite)
dfFormat['vh_notyet_bought'] = ((dfFormat['vh_purchase_date']-dfFormat['submission_created_at'])/ np.timedelta64(1, 'D')>5)


In [1018]:
BaseQuotation['is_max_1_not_responsible'].value_counts()

is_max_1_not_responsible
False    175651
True      29659
Name: count, dtype: int64

In [1019]:

display(dfFormat['md_code_departement'].value_counts())


md_code_departement
59    12447
69    10737
13    10594
77     9128
93     8884
      ...  
15      334
23      241
48      171
99       11
No        3
Name: count, Length: 98, dtype: int64

In [1020]:
# Definition de var_other qui contient toutes les variables qui n'ont pas été explicitement rejetées dans var_suppress
from itertools import chain
var_other = [col for col in dfFormat.columns if col not in list(chain(var_process, var_risk, var_premium ,var_to_suppress,['account_id', 'submission_created_at']))  ]
for col in var_other:
    if col == 'vh_sra_code':print('probleme')
    if col == 'account_id':print('problème')
    if col == 'md_license_country':print(col)
print('liste var_others')
print(var_other)

liste var_others
['policy_period_public_id', 'policy_public_id', 'sf_external_contact_id', 'policy_term_public_id', 'wakam_quote_id', 'wakam_price_test', 'wakam_price_test_perc_applied', 'job_type', 'job_public_id', 'job_close_date', 'issue_date', 'cancellation_date', 'cancellation_nature', 'sf_submission_id', 'sf_opportunity_id', 'original_effective_date', 'period_start', 'period_end', 'period_id', 'creation_date', 'update_date', 'price_computed_at', 'last_policy_period_id', 'product_code', 'policy_term_id', 'risk_type', 'written_date', 'submission_creation_user_id', 'submission_creation_user_email', 'is_from_aggregator', 'is_anonymous', 'session_id', 'utm_source', 'utm_medium', 'utm_campaign', 'utm_term', 'utm_content', 'policy_risk_profile', 'policy_risk_profile_edu', 'has_formulas', 'has_selected_formula', 'account_public_id', 'ph_prefix', 'ph_birthdate', 'md_property_owner_type', 'md_accommodation_type', 'phc_policy_contact_role_public_id', 'phc_sf_external_contact_id', 'md_driver

In [1021]:
# Suppression des doublons et conservation uniquement des variables d'intérêt
''' sortOrder = [True,True] : choose the first quote of a specific account; [True,False] : the last one ''' 
TransformedDataReading = reload(TransformedDataReading)

subf=TransformedDataReading.UniqueRow(dfFormat,sortOrder = [True, True], var_process = var_process,
            var_risk = var_risk ,       
            var_premium = var_premium,var_other = var_other, size = SizeVariablesKeptGW)
 

Suppression of Doublons 
Il existe 283911 lignes avant dédoublonnage
Il existe 254450 lignes lorsqu'on dédoublonne par account_id


In [1022]:
# Base SRA
df_SRA = pd.read_csv(f'{BASE_PATH}/Wakam_vehiculier_auto 04-2023 (dwld PlugPlay).csv', sep=';',low_memory=False)
# On récupère les 3 variables SRA manquantes (classe, poids véhicule vide, puissance DIN)
df_SRA = df_SRA[['SRAIdentifiant','Classe','PoidsASec','PuissanceReelleMaxiDINenCv','VitesseMaxi']]
# Ajout de variables comme le vehicule Ratio
df_SRA = OrnikarDataFormat.FormatSRA(df_SRA,Stat=False)

In [1023]:
#pandas_profiling.ProfileReport(subf, minimal=True)

In [1024]:
# Merge of all the Databases of GuideWire into a unique One
OrnikarDataFormat = reload(OrnikarDataFormat)
#display(df,df_SRA,df_previous_claims,df_previous_insurers,df_previous_terminations,df_previous_license_cancellations_suspensions) 
BaseQuotation = OrnikarDataFormat.mergeBaseGuideWire(dfFormat,df_SRA,df_previous_claims,df_previous_insurers,df_previous_terminations,df_previous_license_cancellations_suspensions,Stat=True)
print('Number of policies UW : '+str(BaseQuotation['tariff_accepted'].sum()))
#pandas_profiling.ProfileReport(BaseQuotation, minimal=True)
BaseQuotation = OrnikarDataFormat.FormatGuideWire(BaseQuotation)
print('Number of policies UW : '+str(BaseQuotation['tariff_accepted'].sum()))

segment marketing


,count
segment,
3. Standard,164781
2. Jeune permis - > 21 ans,45585
1. Jeune permis - 18-21 ans,33813
4. Sans Antécédent,29494
5. Aggravé (CRM > 100),9829
<NA>,405


segment Reinsurer


,count
ReinsurerSegment,
3. Standard,164781
2. Young Driver with No EduScore,58944
4. Interrupted,29494
1b. Young Driver with EduScore (bad),11024
1a. Young Driver with EduScore (Good),9758
3b. Standard - poor CRM,5791
2b.Young Driver - poor CRM,3530
4b. Interrupted - poor CRM,508
<NA>,77


case with NA Segment 405


,segment,md_risk_profile,md_crm_coefficient,md_license_duration,duree_assurance_sur_3ans,md_edu_score,md_age
185,NaN,Unknown,1.0,0.0,0.000000,<NA>,17.0
743,NaN,Unknown,1.0,0.0,0.000000,<NA>,17.0
1304,NaN,Unknown,1.0,0.0,0.000000,<NA>,17.0
3091,NaN,Unknown,1.0,0.0,0.000000,56,17.0
3691,NaN,Unknown,1.0,0.0,0.000000,64,17.0
...,...,...,...,...,...,...,...
263244,NaN,Unknown,1.0,0.0,8.806452,<NA>,17.0
277543,NaN,Unknown,1.0,0.0,8.806452,<NA>,17.0
277544,NaN,Unknown,1.0,0.0,8.806452,<NA>,17.0
281994,NaN,Unknown,1.0,0.0,11.774194,<NA>,17.0


case with NA ReinsurerSegment


,ReinsurerSegment,md_risk_profile,md_crm_coefficient,md_license_duration,duree_assurance_sur_3ans,md_edu_score
4907,NaN,Unknown,NaN,NaN,0.0,<NA>
7550,NaN,Young Driver,NaN,NaN,0.0,<NA>
12879,NaN,Young Driver,NaN,NaN,0.0,<NA>
13452,NaN,Young Driver,NaN,NaN,0.0,<NA>
15310,NaN,Young Driver,NaN,NaN,0.0,<NA>
...,...,...,...,...,...,...
199387,NaN,Young Driver,NaN,NaN,0.0,<NA>
204715,NaN,Young Driver,NaN,NaN,0.0,<NA>
204716,NaN,Young Driver,NaN,NaN,0.0,<NA>
206010,NaN,Young Driver,NaN,NaN,0.0,<NA>


Output Database with 283907 lines and 215 Columns
Number of policies UW : 2724
['submission_created_at', 'job_close_date', 'bind_date', 'issue_date', 'cancellation_date', 'original_effective_date', 'period_start', 'period_end', 'creation_date', 'update_date', 'price_computed_at', 'written_date', 'ph_birthdate', 'md_birthdate', 'md_date_crm_50', 'md_license_date', 'sd_birthdate', 'vh_purchase_date', 'vh_circulation_date', 'vh_insurance_end_date', 'policy_period_start', 'policy_period_end', 'submission_created_at_Timestamp']
Number of policies UW : 2724


In [1025]:
# Indicatrices - Règles GW 
OrnikarDataFormat = reload(OrnikarDataFormat)
BaseQuotation = OrnikarDataFormat.IndicatorsUnderwritingRules(df_temp=BaseQuotation,Stat = True)
print('Number of policies UW : '+str(BaseQuotation['tariff_accepted'].sum()))



Profil Original Guidewire


/Users/marouan.korfed/Desktop/GitHub/2023 Ornikar Pricing/pricing-insurance/Python Official Library/OrnikarDataFormat.py:385: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df_temp['S_autres_dpt_exclus'] = df_temp['is_STANDARD'] & (df_temp['is_dpt_20']|df_temp['is_dpt_96']|df_temp['is_dpt_97']|df_temp['is_dpt_Monaco']|df_temp['is_dpt_99'])
/Users/marouan.korfed/Desktop/GitHub/2023 Ornikar Pricing/pricing-insurance/Python Official Library/OrnikarDataFormat.py:386: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df_temp['S_Age_inf21'

count
policy_risk_profile         is_JEUNE is_STANDARD is_SANS_ANTECEDENT        
Standard                    False    True        False               136460
Young Driver                True     False       False                62642
No Previous Insurance       False    False       True                 23242
Unknown                     True     False       False                15983
                            False    True        False                13980
                                     False       False                13688
Standard                    False    False       True                  5198
Unknown                     False    False       True                  4773
Aggravated Not Paid Premium False    False       False                 4202
Aggravated Malicious        False    False       False                 2042
No Previous Insurance       False    True        False                  596
Standard                    False    False       False                  533
                            True     False       False                  339
No Previous Insurance       True     False       False                  108
Young Driver                False    False       False                   68
No Previous Insurance       False    False       False                   29
Aggravated Alcohol or Drugs False    False       False                   22
Aggravated Malicious        False    False       True                     2

Number of policies UW : 2724


In [1026]:
#display(dfFormatLL[(dfFormatLL['prospect_id_trail'] == 861)])
#display(dfFormatLL[['is_from_aggregator','prospect_id','submission_created_at']][(dfFormatLL['prospect_id']=='1981-05-24_CHEVROLET_CRUZE_09120')])*



# Identification of policy coming from aggregator 

OrnikarDataFormat = reload(OrnikarDataFormat)
dfFormatLL = OrnikarDataFormat.LinkLeLynx(df=BaseQuotation,Stat=True,BASE_PATH_RES=BASE_PATH_RES)

#dfFormatLL = OrnikarDataFormat.LinkLeLynx(df=BaseQuotation[(BaseQuotation['segment']=='1. Jeune permis - 18-21 ans')],Stat=True,BASE_PATH_RES=BASE_PATH_RES)
#display(dfFormatLL[['prospect_id','ph_email','is_anonymous']][(dfFormatLL['prospect_id']=='1996-08-05_FIAT_500_31700')&(dfFormatLL['is_not_anonymous']==True)])
#display(dfFormatLL[['prospect_id','vh_notyet_bought','is_from_aggregator','is_anonymous','has_been_on_Ornikarsite','is_bound','session_id','ph_email','is_anonymous']][(dfFormatLL['prospect_id']=='1981-05-24_CHEVROLET_CRUZE_09120')&(dfFormatLL['is_anonymous']==False)])



#dfFormatLL['clicked'] = (dfFormatLL['session_id'].notna())
#dfFormatLL['is_bound'] = (dfFormatLL['period_status']=='Bound')
#display(dfFormatLL['vh_notyet_bought'].value_counts(dropna=False))



(283907, 395)

(283621, 399)

,prospect_id,counts
0,1981-05-24_CHEVROLET_CRUZE_09120,1159
1,1946-01-01_CITROEN_C5 II TOURER_45000,653
2,1974-10-03_RENAULT_CLIO IV_39300,519
3,1980-11-12_PEUGEOT_308_31820,453
4,1988-10-14_OPEL_AGILA_44640,372
...,...,...
169090,1996-01-23_RENAULT_CLIO V_30540,1
169091,1998-03-15_RENAULT_CLIO V_54000,1
169092,2003-06-22_RENAULT_CLIO V_95370,1
169093,1961-06-16_PEUGEOT_407_69800,1


TotalNumber of Quote Submissions that are done by competition for Pricing Watch : 14500


In [1027]:
dfStat = dfFormatLL[['prospect_id','vh_notyet_bought','is_from_aggregator','is_anonymous','is_not_anonymous','is_bound']].groupby(by = ['prospect_id'],as_index=False).agg(vh_notyet_bought=('vh_notyet_bought', max), is_from_aggregator=('is_from_aggregator', 'max'), has_been_on_aggregator=('is_anonymous', 'max'),
                                                                                                                                                                  has_been_on_Ornikarsite=('is_not_anonymous', 'max'),
                                                                                                                                                                   is_bound=('is_bound', 'max'),SubmissionNbr=('is_bound', 'count'))
#display(dfStat.sort_values(by=['SubmissionNbr'], ascending = False))
#display(dfStat[(dfStat['has_been_on_Ornikarsite'] == 1)&(dfStat['has_been_on_aggregator'] == 1)].sort_values(by=['SubmissionNbr'], ascending = False))

Stat = pd.DataFrame(dfStat[['is_from_aggregator','has_been_on_aggregator','vh_notyet_bought','has_been_on_Ornikarsite','is_bound']].value_counts(dropna=False).reset_index(name='counts'), index=None)
display(Stat)


,is_from_aggregator,has_been_on_aggregator,vh_notyet_bought,has_been_on_Ornikarsite,is_bound,counts
0,True,True,False,False,False,86031
1,False,False,False,True,False,35201
2,True,True,True,False,False,32281
3,False,False,True,True,False,6843
4,True,True,False,True,False,4515
5,False,False,False,True,True,2069
6,True,True,True,True,False,1392
7,True,True,False,True,True,412
8,True,False,False,True,False,125
9,False,False,True,True,True,110


### 2.2 Statistique sur la souscription et sortie Latex

In [1028]:
                     

# segemnts avec faible NbreQuote
# ['prospect_id_trail']
def UWAnalysis(df, date="2023-06-22",Stat=True):

    # Creation of Date, a variable comparing last day result with previous week and since the beginning
    dateJour = pd.to_datetime(date)
    dateWeek = dateJour - np.timedelta64(7, 'D')
    print(dateWeek)
    
    df['Date'] = '3. > 1 week'
    df['Date'] = df['Date'].mask(df['submission_created_at'] > (dateWeek),"2. W Week")
    df['Date'] = df['Date'].mask(df['submission_created_at'] == dateJour,"1. D day")
    df['Date'] = df['Date'].mask(df['submission_created_at'] > dateJour,"0. Post D day")

    print('There are UW after D Day : '+str(df[['Date']][(df['Date']== "0. Post D day")].count()))
    
    df = df[~(df['Date']== "0. Post D day")]

    if Stat: display(df[['Date']].value_counts())
    df['wakam_called'] =1
    for variable in ['wakam_called','tariff_viewed','tariff_accepted','is_MER']:
        Newvariable = variable+'_dedouble'
        Totalvariable = variable+'_Total'
        df = df.merge(df.groupby(by=['prospect_id']).agg(Total=pd.NamedAgg(column=variable, aggfunc="sum")), how='left', on ='prospect_id')
        df[Newvariable] = df[variable] / df['Total']
        df[Newvariable] = df[Newvariable].fillna(value=0)
        #display(df[['prospect_id',variable,Newvariable,'Total','Date']].head(5))
        df = df.drop(columns='Total')
    #display(df['segment'].value_counts())
    df['Young Driver with EduScore (Good)'] = (df['ReinsurerSegment'] == '1a. Young Driver with EduScore (Good)')
    df['Young Driver with EduScore (bad)'] = (df['ReinsurerSegment'] == '1b. Young Driver with EduScore (bad)')
    df['Young Driver with No EduScore'] = (df['ReinsurerSegment'] == '2. Young Driver with No EduScore')
    df['Young Driver - 18-21 year'] = (df['segment'] == '1. Jeune permis - 18-21 ans')
    df['Standard - poor CRM'] = (df['ReinsurerSegment'] == '3b. Standard - poor CRM')
    df['Standard'] = (df['ReinsurerSegment'] == '3. Standard')
    #display(df['Young Driver with EduScore (Good)'].value_counts())

    #Definition ici de is_MER_temp car besoin de la variable 'tariff_viewed_dedouble'
    df['is_MER_temp'] = np.where(df['is_from_aggregator'], df['is_MER'],df['tariff_viewed_dedouble'])

    return df

dfFormatLL2 = UWAnalysis(df=dfFormatLL)
dfFormatLL2.shape

2023-06-15 00:00:00
There are UW after D Day : Date    81078
dtype: int64


Date       
3. > 1 week    146111
2. W Week       43272
1. D day        13160
Name: count, dtype: int64

/var/folders/5v/rlb1sc3s6p7g1d4lvm87clb40000gn/T/ipykernel_14434/145272937.py:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['wakam_called'] =1


(202543, 412)

In [1029]:
def AnalysisbyAxis(df,axis,Variabletoanalyse,title = 'essai'):
    ''' 
    ['NbreQuote', 'PP_Ornikar', 'Top5_Wakam', 'ecart_euros_Bench1', 'prix_Ornikar_TTC_ya'] a rajouter si on travaille avec la base BaseQuotationavecFuret
    '''

    dfStat = pd.DataFrame(df.groupby(by=axis,dropna=False,as_index=False).agg(
                                                            # NbreQuoteOrnikar=pd.NamedAgg(column="prix_Ornikar_TTC_ya", aggfunc=np.count_nonzero),
                                                            # prix_Ornikar_TTC_ya=pd.NamedAgg(column="prix_Ornikar_TTC_ya", aggfunc="mean"),
                                                            SubmittedQuotationsNonDedouble =  pd.NamedAgg(column="wakam_called", aggfunc="count"),       #Wakam a été appelé
                                                            SubmittedQuotations =  pd.NamedAgg(column="wakam_called_dedouble", aggfunc="sum"),       #Wakam a été appelé
                                                            ViewedQuotations =  pd.NamedAgg(column="tariff_viewed_dedouble", aggfunc="sum"), 
                                                            MER =  pd.NamedAgg(column="is_MER_temp", aggfunc="sum"),      #MER
                                                            UWPolicies =  pd.NamedAgg(column="tariff_accepted", aggfunc="sum"),       #polices

                                                            #AxeACP1 = pd.NamedAgg(column="AxeACP1", aggfunc="mean"),
                                                            #AxeACP2 = pd.NamedAgg(column="AxeACP2", aggfunc="mean"),
                                                            #AxeACP3 = pd.NamedAgg(column="AxeACP3", aggfunc="mean"),
                                                            #AxeACP4 = pd.NamedAgg(column="AxeACP4", aggfunc="mean"),
                                                            ))
    

    Total = pd.DataFrame(dfStat.sum()).T
    Total[axis]='Total'
    dfStat = pd.DataFrame(pd.concat([dfStat,Total], axis=0).pivot(columns = axis).sum()).T
 
    #dfStat = dfStat.reset_index(drop=True)
    dfStat['Title'] = title.replace('_', ' ')
 
    
    return dfStat


BaseQuotationbyAxis= AnalysisbyAxis(df=dfFormatLL2,axis='Date',Variabletoanalyse=['md_crm_coefficient'])
#BaseQuotationavecFuretbyAxis= AnalysisbyAxis(df=BaseQuotationavecFuret,axis='WeekAkur8',Variabletoanalyse=['md_crm_coefficient'])
BaseQuotationbyAxis

                                         

SubmittedQuotationsNonDedouble                                \
Date                       1. D day 2. W Week 3. > 1 week   Total   
0                             13160     43272      146111  202543   

     SubmittedQuotations                                        \
Date            1. D day     2. W Week   3. > 1 week     Total   
0            8042.389765  26322.357276  90292.252959  124657.0   

     ViewedQuotations                ...                  MER               \
Date         1. D day     2. W Week  ...    Total    1. D day    2. W Week   
0         4427.472444  15170.228484  ...  69108.0  805.412825  3501.437747   

                                 UWPolicies                              Title  
Date   3. > 1 week         Total   1. D day 2. W Week 3. > 1 week Total         
0     21879.320085  26186.170656         56       262        2012  2330  essai  

[1 rows x 21 columns]

In [1030]:

def LoopOverVariable(df,axis,Variabletoanalyse,title = 'essai'):
    
    
    df['J_vh_notyet_bought'] = df['is_JEUNE'] & df['vh_notyet_bought']
    df['S_vh_notyet_bought'] = df['is_STANDARD'] & df['vh_notyet_bought']
    df['SA_vh_notyet_bought'] = df['is_SANS_ANTECEDENT'] & df['vh_notyet_bought']
    df['SA_vh_notyet_bought'] = df['is_SANS_ANTECEDENT'] & df['vh_notyet_bought']

    df['J_Hors_EduScore_inf_21ans'] = df['Young Driver with No EduScore'] & df['is_md_age_inf_21ans']
    df['J_Hors_EduScore_21_25ans'] = df['Young Driver with No EduScore'] & df['is_md_age_sup_21_inf25ans']
    df['J_Hors_EduScore_sup_25ans'] = df['Young Driver with No EduScore'] & df['is_md_age_sup_25ans']
   
    VariableExclusion = ['is_Exclusion_Geographique', 'is_Exclusion_Conducteur','is_Exclusion_Vehicule', 'is_RAG']
    
    #VariableProcess = ['vh_notyet_bought', 'is_CRM_50']
    
    VariableProfil = ['is_JEUNE','is_STANDARD','is_SANS_ANTECEDENT','is_RAG']

    VariableStandard = ['is_STANDARD','S_CRM_sup1','S_Age_inf21','S_HORS_GC', 'S_Tournee', 'S_vitesse_sup235', 'S_age_vh_sup30', 'S_HORS_histo_sinistres', 'S_resiliation',  'S_dpt_exclus', 'S_permis_hors_UE']
    
    VariableInterrupted = ['is_SANS_ANTECEDENT','SA_CRM_sup1', 'S_Age_inf21', 'SA_HORS_vh_ratio', 'SA_HORS_GC', 'SA_Tournee', 'SA_vitesse_sup235', 'SA_age_vh_sup30', 'SA_HORS_histo_sinistres', 'SA_resiliation', 'SA_dpt_exclus',  'SA_permis_hors_UE' ]

    VariableJEUNE = ['is_JEUNE','J_CRM_sup1', 'J_Age_inf18', 'J_HORS_vh_ratio', 'J_HORS_GC', 'J_Tournee', 'J_vitesse_sup235', 'J_age_vh_sup30', 'J_HORS_histo_sinistres', 'J_resiliation', 'J_dpt_exclus', 'J_permis_hors_UE','Young Driver with EduScore (Good)','Young Driver with EduScore (bad)','Young Driver with No EduScore','Young Driver - 18-21 year']
    
    #VariableRAG =['is_RAG_ALCOOL','is_RAG_MALUS','is_RAG_NPP','is_RAG','is_sup_2_claims']
    
    VariableRAG =['is_RAG', 'is_RAG_ALCOOL','is_RAG_MALUS','is_RAG_NPP']

 
    VariableJEUNE_Exclusion = ['is_JEUNE','J_Exclusion_Geographique', 'J_Exclusion_Conducteur', 'J_Exclusion_Vehicule', 'J_md_age_18_21ans', 'Young Driver with EduScore (Good)','Young Driver with EduScore (bad)', 'J_Hors_EduScore_inf_21ans', 'J_Hors_EduScore_21_25ans', 'J_Hors_EduScore_sup_25ans', 'J_vh_notyet_bought']
 
    VariableSTANDARD_Exclusion = ['is_STANDARD','S_Exclusion_Geographique', 'S_Exclusion_Conducteur', 'S_Exclusion_Vehicule', 'S_CRM_50', 'S_vh_notyet_bought']

    VariableSANSANT_Exclusion = ['is_SANS_ANTECEDENT','SA_Exclusion_Geographique', 'SA_Exclusion_Conducteur', 'SA_Exclusion_Vehicule', 'SA_md_age_inf25ans', 'SA_md_age_sup25ans', 'SA_vh_notyet_bought']
  
  

    VariableJEUNE_Footprint = ['is_JEUNE','J_CRM', 'J_AGE', 'J_AGE_PERMIS', 'J_VH_RATIO', 'J_GROUPE_CLASSE', 'J_AGE_VH_30ANS', 'J_VITESSE_VH_235', 'J_PERMIS_HORS_UE', 'J_DPT_EXCLUS', 'J_TOURNEE']
    VariableSTANDARD_Footprint = ['is_STANDARD','S_CRM', 'S_AGE', 'S_AGE_PERMIS', 'S_VH_RATIO', 'S_GROUPE_CLASSE', 'S_AGE_VH_30ANS', 'S_VITESSE_VH_235', 'S_PERMIS_HORS_UE', 'S_DPT_EXCLUS', 'S_TOURNEE']
    VariableSANSANT_Footprint = ['is_SANS_ANTECEDENT','SA_CRM', 'SA_AGE', 'SA_AGE_PERMIS', 'SA_VH_RATIO', 'SA_GROUPE_CLASSE', 'SA_AGE_VH_30ANS', 'SA_VITESSE_VH_235', 'SA_PERMIS_HORS_UE', 'SA_DPT_EXCLUS', 'SA_TOURNEE']
 

    VariableJEUNEV2 = ['is_JEUNE','J_CRM_sup1', 'J_dpt_13_93', 'J_autres_dpt_exclus', 'J_Age_inf18', 'J_Tournee', 'J_HORS_vh_ratio', 'J_HORS_GC', 'J_vitesse_sup235', 'J_age_vh_sup30', 'J_resiliation', 'J_permis_hors_UE']
    
    VariableSTANDARDV2 = ['is_STANDARD','S_CRM_sup1', 'S_dpt_13_93', 'S_autres_dpt_exclus', 'S_Age_inf21', 'S_Tournee', 'S_HORS_GC', 'S_vitesse_sup235', 'S_age_vh_sup30', 'S_resiliation', 'S_permis_hors_UE']
    
    VariableSANSANTV2 = ['is_SANS_ANTECEDENT','SA_CRM_sup1', 'SA_dpt_13_93', 'SA_autres_dpt_exclus', 'SA_Age_inf21', 'SA_Tournee', 'SA_HORS_vh_ratio', 'SA_HORS_GC', 'SA_vitesse_sup235', 'SA_age_vh_sup30', 'SA_resiliation', 'SA_permis_hors_UE']
    

    CategoryVariableAnalysis = [VariableProfil, VariableJEUNE_Exclusion,VariableSTANDARD_Exclusion, VariableSANSANT_Exclusion, VariableJEUNEV2, VariableSTANDARDV2, VariableSANSANTV2, VariableRAG]# VariableJEUNE, VariableStandard,VariableInterrupted,VariableRAG]

    print(CategoryVariableAnalysis)
    dfStat= AnalysisbyAxis(df=df,axis='Date',Variabletoanalyse=['md_crm_coefficient'],title = 'Total')
    for index, typevariable in enumerate(CategoryVariableAnalysis):
        print(typevariable)
        dfFormatLL3 = df[df[typevariable].any(axis='columns')]
        dfStat= pd.concat([dfStat,AnalysisbyAxis(df=dfFormatLL3,axis='Date',Variabletoanalyse=['md_crm_coefficient'],title = str(index)+'. At least one below conditions')],axis=0)
        
        #[(Test)]
        for variable in typevariable:
            dfFormatLL3 = df[df[variable]]
            dfStat= pd.concat([dfStat,AnalysisbyAxis(df=dfFormatLL3,axis='Date',Variabletoanalyse=['md_crm_coefficient'],title = str(index)+'. '+variable)],axis=0)

        dfFormatLL3 = df[~df[typevariable].any(axis='columns')]
        dfStat= pd.concat([dfStat,AnalysisbyAxis(df=dfFormatLL3,axis='Date',Variabletoanalyse=['md_crm_coefficient'],title = str(index)+'. None above condition respected')],axis=0)
    dfStat = dfStat[[('Title',    ''),
                ('SubmittedQuotationsNonDedouble',    '1. D day'),
                ('SubmittedQuotationsNonDedouble',   '2. W Week'),
                ('SubmittedQuotationsNonDedouble', '3. > 1 week'),
                ('SubmittedQuotationsNonDedouble',       'Total'),
                (           'SubmittedQuotations',    '1. D day'),
                (           'SubmittedQuotations',   '2. W Week'),
                (           'SubmittedQuotations', '3. > 1 week'),
                (           'SubmittedQuotations',       'Total'),
                (              'ViewedQuotations',    '1. D day'),
                (              'ViewedQuotations',   '2. W Week'),
                (              'ViewedQuotations', '3. > 1 week'),
                (              'ViewedQuotations',       'Total'),
                (                           'MER',    '1. D day'),
                (                           'MER',   '2. W Week'),
                (                           'MER', '3. > 1 week'),
                (                           'MER',       'Total'),
                (                    'UWPolicies',    '1. D day'),
                (                    'UWPolicies',   '2. W Week'),
                (                    'UWPolicies', '3. > 1 week'),
                (                    'UWPolicies',       'Total')]]
    
    styler = dfStat.style
    (styler.format(subset="SubmittedQuotationsNonDedouble", precision=0,thousands=',',na_rep='')
      .format(subset=BaseQuotationbyAxis.columns.get_loc_level('1. D day', level=1)[0], precision=0,thousands=',',na_rep='')
      .format(subset=BaseQuotationbyAxis.columns.get_loc_level('2. W Week', level=1)[0], precision=0,thousands=',',na_rep='')
      .format(subset=BaseQuotationbyAxis.columns.get_loc_level('3. > 1 week', level=1)[0], precision=0,thousands=',',na_rep='')
            .format(subset=BaseQuotationbyAxis.columns.get_loc_level('1. D day', level=1)[1], precision=0,thousands=',',na_rep='')
      .format(subset=BaseQuotationbyAxis.columns.get_loc_level('2. W Week', level=1)[1], precision=0,thousands=',',na_rep='')
      .format(subset=BaseQuotationbyAxis.columns.get_loc_level('3. > 1 week', level=1)[1], precision=0,thousands=',',na_rep='')
      .format_index(escape="latex", axis=1)
      .format_index(escape="latex", axis=0)
      .hide(level=0, axis=0))  
    return styler

dfStatOrganique = LoopOverVariable(df=dfFormatLL2[(~dfFormatLL2['is_from_aggregator'])],axis='Date',Variabletoanalyse=['md_crm_coefficient'])
display(dfStatOrganique)
dfStatAggregator = LoopOverVariable(df=dfFormatLL2[(dfFormatLL2['is_from_aggregator'])],axis='Date',Variabletoanalyse=['md_crm_coefficient'])
display(dfStatAggregator)
dfStatTotal = LoopOverVariable(df=dfFormatLL2,axis='Date',Variabletoanalyse=['md_crm_coefficient'])
display(dfStatTotal)


/var/folders/5v/rlb1sc3s6p7g1d4lvm87clb40000gn/T/ipykernel_14434/182446035.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['J_vh_notyet_bought'] = df['is_JEUNE'] & df['vh_notyet_bought']
/var/folders/5v/rlb1sc3s6p7g1d4lvm87clb40000gn/T/ipykernel_14434/182446035.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['S_vh_notyet_bought'] = df['is_STANDARD'] & df['vh_notyet_bought']
/var/folders/5v/rlb1sc3s6p7g1d4lvm87clb40000gn/T/ipykernel_14434/182446035.py:6: SettingWithCopyWarning: 
A value is tryi

[['is_JEUNE', 'is_STANDARD', 'is_SANS_ANTECEDENT', 'is_RAG'], ['is_JEUNE', 'J_Exclusion_Geographique', 'J_Exclusion_Conducteur', 'J_Exclusion_Vehicule', 'J_md_age_18_21ans', 'Young Driver with EduScore (Good)', 'Young Driver with EduScore (bad)', 'J_Hors_EduScore_inf_21ans', 'J_Hors_EduScore_21_25ans', 'J_Hors_EduScore_sup_25ans', 'J_vh_notyet_bought'], ['is_STANDARD', 'S_Exclusion_Geographique', 'S_Exclusion_Conducteur', 'S_Exclusion_Vehicule', 'S_CRM_50', 'S_vh_notyet_bought'], ['is_SANS_ANTECEDENT', 'SA_Exclusion_Geographique', 'SA_Exclusion_Conducteur', 'SA_Exclusion_Vehicule', 'SA_md_age_inf25ans', 'SA_md_age_sup25ans', 'SA_vh_notyet_bought'], ['is_JEUNE', 'J_CRM_sup1', 'J_dpt_13_93', 'J_autres_dpt_exclus', 'J_Age_inf18', 'J_Tournee', 'J_HORS_vh_ratio', 'J_HORS_GC', 'J_vitesse_sup235', 'J_age_vh_sup30', 'J_resiliation', 'J_permis_hors_UE'], ['is_STANDARD', 'S_CRM_sup1', 'S_dpt_13_93', 'S_autres_dpt_exclus', 'S_Age_inf21', 'S_Tournee', 'S_HORS_GC', 'S_vitesse_sup235', 'S_age_vh_sup

/var/folders/5v/rlb1sc3s6p7g1d4lvm87clb40000gn/T/ipykernel_14434/182446035.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['J_vh_notyet_bought'] = df['is_JEUNE'] & df['vh_notyet_bought']
/var/folders/5v/rlb1sc3s6p7g1d4lvm87clb40000gn/T/ipykernel_14434/182446035.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['S_vh_notyet_bought'] = df['is_STANDARD'] & df['vh_notyet_bought']
/var/folders/5v/rlb1sc3s6p7g1d4lvm87clb40000gn/T/ipykernel_14434/182446035.py:6: SettingWithCopyWarning: 
A value is tryi

[['is_JEUNE', 'is_STANDARD', 'is_SANS_ANTECEDENT', 'is_RAG'], ['is_JEUNE', 'J_Exclusion_Geographique', 'J_Exclusion_Conducteur', 'J_Exclusion_Vehicule', 'J_md_age_18_21ans', 'Young Driver with EduScore (Good)', 'Young Driver with EduScore (bad)', 'J_Hors_EduScore_inf_21ans', 'J_Hors_EduScore_21_25ans', 'J_Hors_EduScore_sup_25ans', 'J_vh_notyet_bought'], ['is_STANDARD', 'S_Exclusion_Geographique', 'S_Exclusion_Conducteur', 'S_Exclusion_Vehicule', 'S_CRM_50', 'S_vh_notyet_bought'], ['is_SANS_ANTECEDENT', 'SA_Exclusion_Geographique', 'SA_Exclusion_Conducteur', 'SA_Exclusion_Vehicule', 'SA_md_age_inf25ans', 'SA_md_age_sup25ans', 'SA_vh_notyet_bought'], ['is_JEUNE', 'J_CRM_sup1', 'J_dpt_13_93', 'J_autres_dpt_exclus', 'J_Age_inf18', 'J_Tournee', 'J_HORS_vh_ratio', 'J_HORS_GC', 'J_vitesse_sup235', 'J_age_vh_sup30', 'J_resiliation', 'J_permis_hors_UE'], ['is_STANDARD', 'S_CRM_sup1', 'S_dpt_13_93', 'S_autres_dpt_exclus', 'S_Age_inf21', 'S_Tournee', 'S_HORS_GC', 'S_vitesse_sup235', 'S_age_vh_sup

[['is_JEUNE', 'is_STANDARD', 'is_SANS_ANTECEDENT', 'is_RAG'], ['is_JEUNE', 'J_Exclusion_Geographique', 'J_Exclusion_Conducteur', 'J_Exclusion_Vehicule', 'J_md_age_18_21ans', 'Young Driver with EduScore (Good)', 'Young Driver with EduScore (bad)', 'J_Hors_EduScore_inf_21ans', 'J_Hors_EduScore_21_25ans', 'J_Hors_EduScore_sup_25ans', 'J_vh_notyet_bought'], ['is_STANDARD', 'S_Exclusion_Geographique', 'S_Exclusion_Conducteur', 'S_Exclusion_Vehicule', 'S_CRM_50', 'S_vh_notyet_bought'], ['is_SANS_ANTECEDENT', 'SA_Exclusion_Geographique', 'SA_Exclusion_Conducteur', 'SA_Exclusion_Vehicule', 'SA_md_age_inf25ans', 'SA_md_age_sup25ans', 'SA_vh_notyet_bought'], ['is_JEUNE', 'J_CRM_sup1', 'J_dpt_13_93', 'J_autres_dpt_exclus', 'J_Age_inf18', 'J_Tournee', 'J_HORS_vh_ratio', 'J_HORS_GC', 'J_vitesse_sup235', 'J_age_vh_sup30', 'J_resiliation', 'J_permis_hors_UE'], ['is_STANDARD', 'S_CRM_sup1', 'S_dpt_13_93', 'S_autres_dpt_exclus', 'S_Age_inf21', 'S_Tournee', 'S_HORS_GC', 'S_vitesse_sup235', 'S_age_vh_sup

In [1031]:
# conversion rate
#BaseQuotationbyAxis["Conversion Rate",'1. D day'] = BaseQuotationbyAxis["UWPolicies",'1. D day'] 
#BaseQuotationbyAxis

### Export to Latex & Excel

In [1032]:
#https://pandas.pydata.org/docs/reference/api/pandas.io.formats.style.Styler.to_latex.html
with open(f'{LATEX_DIRECTORY}/output.tex', "w") as f:
  print(dfStatTotal.to_latex(
    caption="From total Policy Number to Underwriting",
    environment ='table*',
    clines="skip-last;data",
    convert_css=True,
    position_float="centering",
    multicol_align="c",
    siunitx = True,
    hrules=True,
)  , file=f)


FileNotFoundError: [Errno 2] No such file or directory: '/Users/marouan.korfed/Desktop/GitHub/2023 Ornikar Pricing/pricing-insurance/Latex/OrnikarInsuranceAutomaticResults/Dashboard_Latex//output.tex'

In [ ]:
with pd.ExcelWriter(f'{BASE_PATH_RES}/output_22062023_V4_VisionFootprint.xlsx') as writer:
    dfStatOrganique.to_excel(writer, na_rep = '',     sheet_name="Flux Organique",)
    dfStatAggregator.to_excel(writer, na_rep = '', sheet_name="Flux Aggregator",)
    dfStatTotal.to_excel(writer, na_rep = '',     sheet_name="Flux Total",)

In [ ]:
# Automatic call of Latex to output pdf computing
# https://tex.stackexchange.com/questions/524098/calling-pdflatex-from-a-python-script
import os  
LATEX_DIRECTORY_RES ='C:/Users/guill/GitHub/2023 Ornikar Pricing/pricing-insurance/Latex/'
#LATEX_DIRECTORY_RES = 'C:\Users\guill\GitHub\2023 Ornikar Pricing\pricing-insurance\Latex'
#LATEX_MAIN_DIRECTORY ='C:\Users\guill\GitHub\2023 Ornikar Pricing\pricing-insurance\Latex\OrnikarInsuranceAutomaticResults'
#os.system(f'pdflatex {LATEX_MAIN_DIRECTORY}/main.tex -synctex=1 -interaction=batchmode -output-directory={LATEX_DIRECTORY_RES}')
os.system(r"pdflatex C:\Users\guill\GitHub\2023 Ornikar Pricing\pricing-insurance\Latex\OrnikarInsuranceAutomaticResults\main.tex -synctex=1 -interaction=batchmode")
# -synctex=1 -interaction=batchmode -output-directory=C:\Users\guill\GitHub\2023 Ornikar Pricing\pricing-insurance\Latex\"
#os.system(f'pdflatex {LATEX_MAIN_DIRECTORY}/main.tex {LATEX_DIRECTORY}')
 

In [ ]:
print(f'pdflatex {LATEX_DIRECTORY} ')


pdflatex C:/Users/guill/GitHub/2023 Ornikar Pricing/pricing-insurance/Latex/OrnikarInsuranceAutomaticResults/Dashboard_Latex/ 


### Ci-dessous : Autres reporting 
1 - ReportingFootprintConversion : 
- donne une vision entre deux dates du volume d'appels tarifaires, footprint et taux de conversion (et cela par variable étudiée)

2 - ReportingRepartitionMIX : 
- donne une vision de la répartition du MIX entre deux dates (et cela par variable étudiée)

3 - ReportingEmpreinte'PROFIL'Comparaison :
- donne une vision de l'empreinte avec les règles qui éliminent au fur et à mesure

Axes de developpement (23-06-2023): 
- s'assurer de l'alignement des définitions de footprint / taux de conversion avec le tableau de synthèse 'LoopOverVariable' 
- certaines règles de souscriptions ne semblent pas correctement codées (historique de sinistralité notamment / Corse)
- Faire une boucle pour automatiser les reportings dans Excel 
- Faire une boucle pour automatiser les reportings dans Latex

In [ ]:
#variables = ["policy_risk_profile", "categorie_md_age", "categorie_vh_age", "categorie_md_license_seniority", "categorie_md_crm_coefficient"]  
OrnikarDataFormat = reload(OrnikarDataFormat)
dfStat_FtConv = OrnikarDataFormat.ReportingFootprintConversion("2023-06-20", "2023-06-14", dfFormatLL2, 'policy_risk_profile')
dfStat_MIX = OrnikarDataFormat.ReportingRepartitionMIX("2023-06-20", "2023-06-14", dfFormatLL2, 'categorie_md_age')

display(dfStat_FtConv)
display(dfStat_MIX)

,policy_risk_profile,App. Tarifaires J,App. Tarif. J-7,Footprint (%) J,Ft J-7,Tx conversion (%) J,Tx conv. J-7
0,Aggravated Malicious,37.0,26,NaN,NaN,NaN,NaN
1,Aggravated Not Paid Premium,62.0,58,NaN,NaN,NaN,NaN
2,No Previous Insurance,328.0,283,53.0,52.0,3.0,8.0
3,Standard,2214.0,1884,72.0,72.0,1.0,2.0
4,Unknown,648.0,550,NaN,NaN,NaN,NaN
5,Young Driver,663.0,581,77.0,73.0,4.0,6.0
6,Aggravated Alcohol or Drugs,NaN,1,NaN,NaN,NaN,NaN
7,Total,3952.0,3383,58.0,57.0,2.0,3.0


,categorie_md_age,App. Tarifaires J (%),App. Tarif. J-7 (%),Tarifs Vus J (%),Tarifs Vus J-7 (%),Contrats J (%),Contrats J-7 (%)
0,18-21 ans,14.0,15.0,12.0,14.0,16.0,11.0
1,22-30 ans,29.0,31.0,28.0,30.0,32.0,33.0
2,31-50 ans,38.0,36.0,39.0,39.0,41.0,48.0
3,>50 ans,19.0,17.0,22.0,18.0,11.0,8.0
4,Total,100.0,100.0,100.0,100.0,100.0,100.0


In [ ]:
OrnikarDataFormat = reload(OrnikarDataFormat)
dfStat_EmpreinteJEUNEComparaison = OrnikarDataFormat.ReportingEmpreinteJEUNEComparaison('2023-06-18','2023-06-11', dfFormatLL2)
dfStat_EmpreinteSANSANTComparaison = OrnikarDataFormat.ReportingEmpreinteSANSANTComparaison('2023-06-18','2023-06-11', dfFormatLL2)
dfStat_EmpreinteSTANDARDComparaison = OrnikarDataFormat.ReportingEmpreinteSTANDARDComparaison('2023-06-18','2023-06-11', dfFormatLL2)

display(dfStat_EmpreinteJEUNEComparaison)
display(dfStat_EmpreinteSTANDARDComparaison)
display(dfStat_EmpreinteSANSANTComparaison)

,0,1,2,3
Date,2023-06-18,2023-06-18,2023-06-11,2023-06-11
Nombre de profils JEUNE,1400,100,1416,100
CRM <= 1,0,0,0,0
Age permis <3 ans,0,0,0,0
Age conducteur >=18 ans,-13,-1,-5,0
Véhicule ratio,-109,-8,-117,-8
Groupe/Classe,-131,-9,-154,-11
Pas de véhicule sup 30 ans,-6,0,-8,-1
Pas de véhicule avec vitesse >235,0,0,-1,0
Historique sinistralité,-1121,-80,-1094,-77


,0,1,2,3
Date,2023-06-18,2023-06-18,2023-06-11,2023-06-11
Nombre de profils STANDARD,2622,100,3286,100
CRM <= 1,0,0,0,0
Age permis >=3 ans,0,0,0,0
Age conducteur >=21 ans,-6,0,-6,0
Véhicule ratio,0,0,0,0
Groupe/Classe,-231,-9,-247,-8
Historique d'assurance >= 9 mois,0,0,0,0
Historique sinistralité,-1902,-73,-2475,-75
Règle pas de résiliation assureur (ou pour raison autre OK),-467,-18,-550,-17


,0,1,2,3
Date,2023-06-18,2023-06-18,2023-06-11,2023-06-11
Nombre de profils SANS ANTECEDENT,481,100,688,100
CRM <= 1,0,0,0,0
Age permis >=3 ans,0,0,0,0
Age conducteur >=21 ans,-3,-1,-2,0
Véhicule ratio,-32,-7,-28,-4
Groupe/Classe,-12,-2,-20,-3
Pas de véhicule sup 30 ans,-3,-1,-8,-1
Pas de véhicule avec vitesse >235,-8,-2,-10,-1
Historique d'assurance < 9 mois,0,0,0,0
